# Research Track 2 : Assignment2
The objective of this task is to create a Jupyter Notebook that constructs a controller for a Robot operating within an environment. The Notebook will primarily consist of a User Interface, featuring the following components:
1. Motion Control Buttons: These buttons enable the user to control the robot's movements within the environment.
2. Position and Target Plot: This plot visually represents the positions of both the robot and the targets within the environment.
3. Distance Display: A text box will display the distance to the nearest obstacle.
4. Target Achievement Plot: This plot illustrates the number of targets reached versus the number of targets not yet reached by the robot.

In the first cell, all the required modules are imported, and several global variables are initialized.

In [ ]:
import rospy
import actionlib
import ipywidgets as widgets
from IPython.display import display
from sensor_msgs.msg import LaserScan
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets
import matplotlib.pyplot as plt
from nav_msgs.msg import Odometry
from matplotlib.animation import FuncAnimation
from geometry_msgs.msg import PoseStamped 
import actionlib.msg 
import assignment_2_2022.msg 
import os

%matplotlib widget


reached_goal_counter = 0
canceled_goal_counter = 0

In this cell I implemented some buttons for handling the motion of the robot in the environment:

In [ ]:
def send_target(x, y):
    rospy.init_node('NodeA')
    client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)  
    client.wait_for_server()

    x_position = int(x)
    y_position = int(y)
    goal = PoseStamped()


    goal.pose.position.x = x_position
    goal.pose.position.y = y_position


    goal = assignment_2_2022.msg.PlanningGoal(goal)    
    client.send_goal(goal)
    client.wait_for_result()

def set_target_coordinates(button):
    x = int(x_text.value)
    y = int(y_text.value)
    send_target(x, y)

# Create the button and input fields
x_text = widgets.Text(description='X:')
y_text = widgets.Text(description='Y:')
button = widgets.Button(description='Set Target')

# Attach the button callback function
button.on_click(set_target_coordinates)

# Display the widgets
display(x_text, y_text, button)

In this cell, a plot is generated to display the positions of both the robot and the target within the environment. 

In [ ]:
plt.switch_backend('TKAgg')

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.goal_ln, = plt.plot([], [], 'go')  # Line for goal position
        self.x_data, self.y_data = [], []
        self.goal_x, self.goal_y = None, None

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln, self.goal_ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def goal_callback(self, msg):
        self.goal_x = msg.x
        self.goal_y = msg.y

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.goal_ln.set_data(self.goal_x, self.goal_y)
        return self.ln, self.goal_ln


#rospy.init_node('odom_visualizer_node')
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
des_pos_x = rospy.get_param("/des_pos_x")
des_pos_y = rospy.get_param("/des_pos_y")
vis.goal_x = des_pos_x
vis.goal_y = des_pos_y
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)


In this cell, the distance of the closest obstacle is displayed. 

In [ ]:
class obstacle:
    def __init__(self):
        self.distance_text = widgets.FloatText(description='Closest obstacle:', value=5)

    def laser_callback(self, msg):
        closest_distance = min(msg.ranges)
        self.distance_text.value = closest_distance

vis = obstacle()
sub = rospy.Subscriber('/scan', LaserScan, vis.laser_callback)

# Display the distance text box
display(vis.distance_text)

In this cell, the number of reached and canceled targets is presented. 

In [ ]:
plt.switch_backend('TKAgg')


def callback_subscriber(msg):
    global reached_goal_counter, canceled_goal_counter
    
    if msg.status.status == 2:
        canceled_goal_counter += 1
    elif msg.status.status == 3:
        reached_goal_counter += 1


rospy.Subscriber("/reaching_goal/result", assignment_2_2022.msg.PlanningActionResult, callback_subscriber)


x_data = ['Reached Goals', 'Canceled Goals']
y_data = [reached_goal_counter, canceled_goal_counter]
# Plot the data
plt.bar(x_data, y_data)
plt.xlabel('Goal Status')
plt.ylabel('Count')
plt.title('Reached and Canceled Goals')
plt.show()
